In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [ ]:
class HAM10000Dataset(Dataset):
    def __init__(self, csv_file, img_dirs, transform=None):
        self.data = pd.read_csv(csv_file)
        self.img_dirs = img_dirs
        self.transform = transform
        self.label_map = {label: idx for idx, label in enumerate(self.data['dx'].unique())}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]['image_id'] + '.jpg'
        img_path = None
        for dir in self.img_dirs:
            if os.path.exists(os.path.join(dir, img_name)):
                img_path = os.path.join(dir, img_name)
                break
        if img_path is None:
            raise FileNotFoundError(f"Image {img_name} not found in any of the specified directories.")
        
        image = Image.open(img_path).convert('RGB')
        label = self.label_map[self.data.iloc[idx]['dx']]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [ ]:
# Setting the Metadata Path
metadata_path = '../input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv'

# Loading the Metadata
metadata = pd.read_csv(metadata_path)

# Checking the Number of Unique Images
print(f'Total number of images: {len(metadata)}')

In [ ]:
# Splitting the metadata into training and testing sets
train_metadata, test_metadata = train_test_split(metadata, test_size=0.99, random_state=42)

# Saving the split metadata to CSV files
train_metadata.to_csv('train_metadata.csv', index=False)
test_metadata.to_csv('test_metadata.csv', index=False)

In [ ]:
# Defining Image Directories
img_dirs = [
    '../input/skin-cancer-mnist-ham10000/HAM10000_images_part_1',
    '../input/skin-cancer-mnist-ham10000/HAM10000_images_part_2'
]

# Purpose of Image Directories
# These directories are likely part of the dataset's structure, where the images have been split into multiple parts for organizational purposes.
# By listing these directories, the code can later iterate through them to locate and load the images as needed.

# This setup is crucial for managing and accessing the image files efficiently.
# By specifying the directories in a list, the code can easily handle the images regardless of their distribution across multiple folders.
# This approach simplifies the process of loading and preprocessing the images for further analysis or model training.

In [ ]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize images to 64x64
    transforms.ToTensor(),        # Convert to PyTorch tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])

In [ ]:
# Creating Datasets
train_dataset = HAM10000Dataset(csv_file='train_metadata.csv', img_dirs=img_dirs, transform=transform)
test_dataset = HAM10000Dataset(csv_file='test_metadata.csv', img_dirs=img_dirs, transform=transform)

# Creating DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False, num_workers=2)

# Printing Dataset Sizes
print(f'Training dataset size: {len(train_dataset)}')
print(f'Testing dataset size: {len(test_dataset)}')

In [ ]:
# Importing Matplotlib
import matplotlib.pyplot as plt

# Loading a Batch of Images
images, labels = next(iter(train_loader))

# Printing the shapes of the image and label batches
print(f'Image batch shape: {images.shape}')
print(f'Label batch shape: {labels.shape}')

# Creating a Figure for Display
fig, axes = plt.subplots(1, 4, figsize=(12, 4))

# Displaying the First 4 Images
for i in range(4):
    axes[i].imshow(images[i].permute(1, 2, 0).numpy() * 0.5 + 0.5)
    axes[i].set_title(f"Label: {labels[i].item()}")
    axes[i].axis("off")

# Showing the Figure
plt.show()

In [ ]:
import torch

import torch.nn as nn

# Self-Attention Class
class SelfAttention(nn.Module):
    def __init__(self, in_dim):
        super(SelfAttention, self).__init__()
        self.query_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels=in_dim, out_channels=in_dim, kernel_size=1)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        batch_size, C, width, height = x.size()
        proj_query = self.query_conv(x).view(batch_size, -1, width * height).permute(0, 2, 1)
        proj_key = self.key_conv(x).view(batch_size, -1, width * height)
        energy = torch.bmm(proj_query, proj_key)
        attention = torch.softmax(energy, dim=-1)
        proj_value = self.value_conv(x).view(batch_size, -1, width * height)
        out = torch.bmm(proj_value, attention.permute(0, 2, 1))
        out = out.view(batch_size, C, width, height)
        out = self.gamma * out + x
        return out

# Residual Block Class
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(in_channels)
        )

    def forward(self, x):
        return x + self.block(x)

# Generator Class
class Generator(nn.Module):
    def __init__(self, latent_dim, img_channels, img_size):
        super(Generator, self).__init__()
        self.init_size = img_size // 4
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.ReLU(inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, img_channels, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

        self.res_block = ResidualBlock(128)
        self.attention = SelfAttention(128)

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        out = self.res_block(out)
        out = self.attention(out)
        img = self.conv_blocks(out)
        return img

# Instantiating and Testing the Generator
latent_dim = 100
img_channels = 3
img_size = 64

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator(latent_dim, img_channels, img_size).to(device)

# Generate a batch of images
z = torch.randn(4, latent_dim).to(device)
generated_images = generator(z)

# Print the shape of the generated images
print(f'Generated images shape: {generated_images.shape}')